In [1]:
#ECE6885 WH6 6.1 Practice Problem#
# Alper Emin Cetinkaya G45307115 #

In [2]:
import torch
from torchvision import models, transforms
from PIL import Image
import torchvision
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np

In [3]:
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("Ahmed9275/Vit-Cifar100")
model = AutoModelForImageClassification.from_pretrained("Ahmed9275/Vit-Cifar100")

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


config.json:   0%|          | 0.00/4.68k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/344M [00:00<?, ?B/s]

In [4]:
model.eval()

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [5]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [6]:
cifar100_test = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=preprocess)
cifar100_test_loader = DataLoader(
        cifar100_test, shuffle=True, num_workers=4, batch_size=32)

100%|██████████| 169001437/169001437 [00:04<00:00, 34448777.65it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
device = "cuda"

In [8]:
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [9]:
def evaluate_model(model, dataloader, device):
    correct_predictions = 0
    total_samples = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.logits, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = correct_predictions / total_samples

    return accuracy, all_labels, all_predictions

In [10]:
accuracy, all_labels, all_predictions = evaluate_model(model, cifar100_test_loader, device)

In [11]:
accuracy

0.8127

In [12]:
conf_matrix = confusion_matrix(all_labels, all_predictions)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[94  0  1 ...  0  0  0]
 [ 0 90  0 ...  0  0  0]
 [ 0  0 73 ...  0  0  0]
 ...
 [ 0  0  0 ... 82  0  0]
 [ 0  0  2 ...  0 74  0]
 [ 0  0  1 ...  0  0 92]]


In [13]:
len(conf_matrix)

100

In [14]:
class_accuracy = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)

In [15]:
class_accuracy

array([0.94, 0.9 , 0.73, 0.63, 0.56, 0.89, 0.76, 0.87, 0.98, 0.9 , 0.6 ,
       0.76, 0.84, 0.77, 0.78, 0.88, 0.87, 0.91, 0.84, 0.84, 0.96, 0.82,
       0.88, 0.91, 0.9 , 0.8 , 0.79, 0.6 , 0.9 , 0.78, 0.73, 0.92, 0.71,
       0.79, 0.89, 0.54, 0.81, 0.71, 0.81, 0.93, 0.86, 0.9 , 0.8 , 0.86,
       0.71, 0.64, 0.69, 0.69, 0.93, 0.89, 0.72, 0.87, 0.41, 0.92, 0.94,
       0.57, 0.88, 0.93, 0.92, 0.58, 0.84, 0.91, 0.83, 0.81, 0.68, 0.88,
       0.85, 0.82, 0.97, 0.92, 0.79, 0.86, 0.68, 0.79, 0.66, 0.92, 0.94,
       0.79, 0.68, 0.84, 0.79, 0.82, 0.96, 0.81, 0.83, 0.86, 0.87, 0.91,
       0.84, 0.92, 0.72, 0.94, 0.76, 0.71, 0.95, 0.77, 0.73, 0.82, 0.74,
       0.92])

In [16]:
cifar100_labels = [
'apple',
'aquarium_fish',
'baby',
'bear',
'beaver',
'bed',
'bee',
'beetle',
'bicycle',
'bottle',
'bowl',
'boy',
'bridge',
'bus',
'butterfly',
'camel',
'can',
'castle',
'caterpillar',
'cattle',
'chair',
'chimpanzee',
'clock',
'cloud',
'cockroach',
'couch',
'crab',
'crocodile',
'cup',
'dinosaur',
'dolphin',
'elephant',
'flatfish',
'forest',
'fox',
'girl',
'hamster',
'house',
'kangaroo',
'computer_keyboard',
'lamp',
'lawn_mower',
'leopard',
'lion',
'lizard',
'lobster',
'man',
'maple_tree',
'motorcycle',
'mountain',
'mouse',
'mushroom',
'oak_tree',
'orange',
'orchid',
'otter',
'palm_tree',
'pear',
'pickup_truck',
'pine_tree',
'plain',
'plate',
'poppy',
'porcupine',
'possum',
'rabbit',
'raccoon',
'ray',
'road',
'rocket',
'rose',
'sea',
'seal',
'shark',
'shrew',
'skunk',
'skyscraper',
'snail',
'snake',
'spider',
'squirrel',
'streetcar',
'sunflower',
'sweet_pepper',
'table',
'tank',
'telephone',
'television',
'tiger',
'tractor',
'train',
'trout',
'tulip',
'turtle',
'wardrobe',
'whale',
'willow_tree',
'wolf',
'woman',
'worm',
]

In [17]:
class_accuracies = {}

In [18]:
for acc, cls in zip(class_accuracy, cifar100_labels):
  class_accuracies[cls] = acc

In [19]:
class_accuracies = dict(sorted(class_accuracies.items(), key=lambda item: item[1]))

In [20]:
class_accuracies

{'oak_tree': 0.41,
 'girl': 0.54,
 'beaver': 0.56,
 'otter': 0.57,
 'pine_tree': 0.58,
 'bowl': 0.6,
 'crocodile': 0.6,
 'bear': 0.63,
 'lobster': 0.64,
 'shrew': 0.66,
 'possum': 0.68,
 'seal': 0.68,
 'snake': 0.68,
 'man': 0.69,
 'maple_tree': 0.69,
 'flatfish': 0.71,
 'house': 0.71,
 'lizard': 0.71,
 'turtle': 0.71,
 'mouse': 0.72,
 'train': 0.72,
 'baby': 0.73,
 'dolphin': 0.73,
 'willow_tree': 0.73,
 'woman': 0.74,
 'bee': 0.76,
 'boy': 0.76,
 'tulip': 0.76,
 'bus': 0.77,
 'whale': 0.77,
 'butterfly': 0.78,
 'dinosaur': 0.78,
 'crab': 0.79,
 'forest': 0.79,
 'rose': 0.79,
 'shark': 0.79,
 'snail': 0.79,
 'squirrel': 0.79,
 'couch': 0.8,
 'leopard': 0.8,
 'hamster': 0.81,
 'kangaroo': 0.81,
 'porcupine': 0.81,
 'sweet_pepper': 0.81,
 'chimpanzee': 0.82,
 'ray': 0.82,
 'streetcar': 0.82,
 'wolf': 0.82,
 'poppy': 0.83,
 'table': 0.83,
 'bridge': 0.84,
 'caterpillar': 0.84,
 'cattle': 0.84,
 'plain': 0.84,
 'spider': 0.84,
 'tiger': 0.84,
 'raccoon': 0.85,
 'lamp': 0.86,
 'lion': 0.86

In [26]:
conf_matrix_ = conf_matrix - np.eye(len(conf_matrix))*conf_matrix.max()

In [37]:
flat_indices = np.argsort(conf_matrix_.flatten())[-20:]

row_indices, col_indices = np.unravel_index(flat_indices, conf_matrix.shape)

greatest_20_indices = list(zip(row_indices, col_indices))

In [38]:
greatest_20_indices

[(98, 35),
 (36, 65),
 (11, 46),
 (50, 65),
 (30, 73),
 (13, 81),
 (88, 42),
 (35, 11),
 (35, 98),
 (95, 73),
 (55, 72),
 (59, 96),
 (46, 11),
 (2, 11),
 (35, 2),
 (52, 96),
 (47, 96),
 (10, 61),
 (78, 99),
 (52, 47)]

In [39]:
for indices in greatest_20_indices:
  print("Prediction:  " + cifar100_labels[indices[0]] + " : " + cifar100_labels[indices[1]])

Prediction:  woman : girl
Prediction:  hamster : rabbit
Prediction:  boy : man
Prediction:  mouse : rabbit
Prediction:  dolphin : shark
Prediction:  bus : streetcar
Prediction:  tiger : leopard
Prediction:  girl : boy
Prediction:  girl : woman
Prediction:  whale : shark
Prediction:  otter : seal
Prediction:  pine_tree : willow_tree
Prediction:  man : boy
Prediction:  baby : boy
Prediction:  girl : baby
Prediction:  oak_tree : willow_tree
Prediction:  maple_tree : willow_tree
Prediction:  bowl : plate
Prediction:  snake : worm
Prediction:  oak_tree : maple_tree
